<a href="https://colab.research.google.com/github/Sushmitha-Katti/ERA/blob/main/Session_6-Backpropagation_and_Advanced_Architectures/Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [42]:
class Net(nn.Module):
    def __init__(self):
            super(Net, self).__init__()

            self.conv1 = nn.Sequential(                      
            
            nn.Conv2d(1, 16, 3, bias = False),
            nn.ReLU(),
            nn.Dropout2d(0.1),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),


            nn.Conv2d(16, 16, 3, bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            nn.Conv2d(16, 16, 3, bias = False ),        
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 16, 3, bias = False), 
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            

            nn.Conv2d(16, 16, 3, bias = False ), 
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),

            nn.Conv2d(16, 16, 3, bias = False ), 
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),

            nn.Conv2d(16, 32, 3,  bias = False ), 
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),

            nn.AdaptiveAvgPool2d(1),  # Apply GAP
          

           
            
          )
            self.fc = nn.Linear(32, 10)
          


    def forward(self, x):
        x = self.conv1(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        x = F.log_softmax(x, dim=1)
        return x

In [43]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
         Dropout2d-3           [-1, 16, 26, 26]               0
       BatchNorm2d-4           [-1, 16, 26, 26]              32
         Dropout2d-5           [-1, 16, 26, 26]               0
            Conv2d-6           [-1, 16, 24, 24]           2,304
              ReLU-7           [-1, 16, 24, 24]               0
       BatchNorm2d-8           [-1, 16, 24, 24]              32
         Dropout2d-9           [-1, 16, 24, 24]               0
           Conv2d-10           [-1, 16, 22, 22]           2,304
             ReLU-11           [-1, 16, 22, 22]               0
      BatchNorm2d-12           [-1, 16, 22, 22]              32
    

In [44]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [45]:

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch = {epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [46]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch = 1 loss=0.09474251419305801 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.18it/s]



Test set: Average loss: 0.0470, Accuracy: 9867/10000 (98.67%)



epoch = 2 loss=0.07185515016317368 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.96it/s]



Test set: Average loss: 0.0337, Accuracy: 9891/10000 (98.91%)



epoch = 3 loss=0.21066176891326904 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 23.09it/s]


...


Test set: Average loss: 0.0181, Accuracy: 9947/10000 (99.47%)



epoch = 18 loss=0.009561813436448574 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.14it/s]



Test set: Average loss: 0.0174, Accuracy: 9943/10000 (99.43%)



epoch = 19 loss=0.0029094850178807974 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.23it/s]



Test set: Average loss: 0.0183, Accuracy: 9943/10000 (99.43%)

